<a href="https://colab.research.google.com/github/mrdo19111911/3D-Loading-Divide-space-2.0/blob/main/Class_3D_loading_Untitled7_refactor_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 import

In [ ]:
# trả ra kết quả 3D loading




# ITEM_L_ID  =  0
# ITEM_W_ID  =  1
# ITEM_H_ID  =  2
# ITEM_NUM  =  3
# ITEM_ROTATE =  4
# ITEM_POS_L_ID  = 5
# ITEM_POS_W_ID  =  6
# ITEM_POS_H_ID  = 7
# ITEM_ID  =  8

# Main

In [ ]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numba
from numba import njit
import random as rd
import time
import graphviz

import plotly.graph_objects as go
import numpy as np

from numba.typed import List
%autosave 60


Autosaving every 60 seconds


In [ ]:


@njit (cache= True)
def compute_number_of_fit_with_rotation(bin_size,item_size):  #M_lwh, M_l, M_W, M_h, best_rotate
    best_num_item  =  -3
    best_rotation = -1
    # infor_of_best_fit  =   np.empty((4), np.int_)
    
    result  =   np.empty((5), np.int_)
    OZ_rotation_lock = item_size[9]
    number_of_posible_rotation = 6
    if OZ_rotation_lock != 0:
        number_of_posible_rotation = 2

    for rotation in range(number_of_posible_rotation):
        infor_of_fit  =  compute_number_of_fit( bin_size, compute_rotate_item_size(item_size[0:3],rotation, OZ_rotation_lock))
        num_item =  infor_of_fit[0]
        if num_item>best_num_item:
            best_num_item  =   num_item
            best_rotation  =  rotation
            result[0]  =   num_item

            result[1] =  infor_of_fit[1]
            result[2] =  infor_of_fit[2]
            result[3] =  infor_of_fit[3]
            result[4] =  best_rotation

            # result[5]  =  best_rotation
            # print(result)
        #     M_lwh, M_l, M_w, M_h, best_rotate =  temp[0], temp[1], temp[2], temp[3]
    return result

@njit(cache= True)
def compute_box(bin_size, item_size, best_fit, item_size_rotate):#decompose box, input: [size,position]
    # print (bin_size)
    box =  np.empty((6, len(bin_size)), np.int_)
    for i in range(6):
        box[i]= np.copy(bin_size)

    M_lwh, M_l, M_w, M_h, best_rotate  =  best_fit[0], best_fit[1], best_fit[2], best_fit[3], best_fit[4]
    # item_size_rotate  =  compute_rotate_item_size(item_size, best_rotate)
    item_l  =  item_size_rotate[0]
    item_w  =  item_size_rotate[1]
    item_h  =  item_size_rotate[2]
    item_num  =  item_size_rotate[3]

    # print(item_size_rotate)
    # print(best_fit)
    if M_lwh == 0:
        return box
    bin_l  =  bin_size[0]
    bin_w =  bin_size[1]
    bin_h  = bin_size[2]
    bin_pos_l  =  bin_size[3]
    bin_pos_w =  bin_size[4]
    bin_pos_h  = bin_size[5]
    # print("1")
    temp_num  =  item_num
    if temp_num>M_lwh:
        temp_num =  M_lwh
    OQ =  bin_l
    N_l  =  math.ceil(temp_num/(M_w* M_h)) #8
    num_item_front = temp_num- (N_l-1)*M_w* M_h #20-2*4*2
    OF  =  N_l*item_l
    # print(N_l)
    # print('need to compute')
    HI  =  M_w*item_w
    IJ  =  bin_w-HI
    JP  =   math.floor(temp_num/(M_w* M_h))*item_l
    num_colum_face =  math.ceil(num_item_front/M_h)
    FA  = num_colum_face*item_w
    AB  =  (M_h-(num_colum_face*M_h-num_item_front))*item_h
    num_box_AB  =   num_item_front%(M_h)
    FC =  math.floor(num_item_front/M_h )*item_w
    CA  =  FA-FC

    # print(num_box_AB)
    # print("OF", OF,"FC", FC,"CA","FA",FA, CA, "AB", AB)
    FD =  M_h*item_h
    box[0][0:7]  =  [ bin_l-OF, bin_w, bin_h,bin_pos_l+ OF, bin_pos_w, bin_pos_h,0]
    box[1][0:7]   =  [ item_l, bin_w-FA, bin_h, bin_pos_l + OF- item_l,bin_pos_w+ FA, bin_pos_h,1]
    box[2][0:7]   =  [ item_l, FA-FC, bin_h- AB, bin_pos_l+ OF-item_l, bin_pos_w + FC,AB+bin_pos_h,2]

    box[3] [0:7]  =  [ item_l, FC, bin_h- FD, bin_pos_l + OF- item_l, bin_pos_w, FD+bin_pos_h,3]


    box[4] [0:7]  =  [ OF- item_l, bin_w-HI, bin_h, bin_pos_l+0,bin_pos_w+HI, bin_pos_h,4]
    box[5] [0:7]  =  [OF- item_l, HI, bin_h-FD, bin_pos_l+0, bin_pos_w, FD +bin_pos_h,5]


    return box

def move_item_to_prepack(bin_size, item_size,number_of_best_fit, pre_packed_item):
    # print("\t\t",  "pre_packed_item", pre_packed_item)
    item_size_pack= np.copy(item_size)
    item_size_pack[3] = min(item_size[3] , number_of_best_fit[0])
    item_size[3]= item_size[3]- item_size_pack[3]
    pre_packed_item.append([bin_size, item_size_pack, item_size_pack[3], number_of_best_fit])
 


class Helper:
    def __init__(self):
        pass

    @staticmethod
    @njit (cache= True)
    # Cần viết lại hàm này., 
    def decompose_bin(bin_size, item_size) :
        best_fit  =  compute_number_of_fit_with_rotation(bin_size,item_size)
        # number of item  item_list[i][3] 
        # print(best_fit)
        OZ_rotation_lock= item_size[9]

        new_rotate_size  =  compute_rotate_item_size(item_size,best_fit[4], OZ_rotation_lock )
        M_lwh =  new_rotate_size[0]
        M_l = new_rotate_size[1]
        M_w =  new_rotate_size[2]
        M_h =  new_rotate_size[3] 
        best_rotate =  new_rotate_size[4]
        # print(new_rotate_size)
        # compute_rotate_item_size(item_size, best_rotate)    
        boxes  =  compute_box(bin_size, item_size, best_fit,new_rotate_size )
        if item_size[3] == 0:
            # print('emtp')
            boxes =  np.empty((6, len(bin_size)), np.int_)
            for i in range(6):

                for j in range(len(bin_size)):
                    if i==0:
                        boxes[0][j] = bin_size[j]
                    else:
                        boxes[i][j]=0
            boxes[0] = bin_size
        return boxes

helper = Helper()

In [ ]:
@njit (cache= True)
def compute_rotate_item_size(size, rotate, OZ_rotation_lock): # return newsize after rotate. =0 1
    ROTATE_FIX =  np.array([  [0,1,2],  [1,0,2], [0,2,1], [1,2,0], [2,1,0],[2,0,1]], np.int_)
    ROTATE_FIX_WITH_OZ_LOCK =  np.array([  [0,1,2],  [1,0,2]], np.int_)
    # OZ_rotation_lock = size[9]
    if OZ_rotation_lock ==0:

        new  =   ROTATE_FIX[rotate]
    else:

        new  =   ROTATE_FIX_WITH_OZ_LOCK[rotate]
    # aa  =   [size[new[0]], size[new[1]], size[new[2]]]
    new_size  =   np.concatenate((np.array([size[new[0]], size[new[1]], size[new[2]]], np.int_) , size[3:]   ))
    return new_size
@njit (cache= True)
def compute_number_of_fit(bin_size,item_size): #find number of fit of an item without rotate.
    M_l = math.floor(bin_size[0]/item_size[0])
    M_w = math.floor(bin_size[1]/item_size[1])
    M_h = math.floor(bin_size[2]/item_size[2])
    result  =  np.empty((4), np.int_)
    result[0] =  M_h*M_l*M_w
    result[1] =  M_l
    result[2] =  M_w
    result[3] =  M_h
    return result

@njit (cache= True)
def insert___(bin_size, item_size, mylist, number_item, number_of_best_fit):
    # if bin_size[3] ==3790:
    #     print("fuck")
    ITEM_L_ID  =  0
    ITEM_W_ID  =  1
    ITEM_H_ID  =  2

    ITEM_NUM  =  3
    ITEM_ROTATE =  4

    ITEM_POS_L_ID  = 5
    ITEM_POS_W_ID  =  6
    ITEM_POS_H_ID  = 7

    BIN_L_ID  =  0
    BIN_W_ID  =  1
    BIN_H_ID  =  2
    BIN_POS_L_ID  = 3
    BIN_POS_W_ID  =  4
    BIN_POS_H_ID  =  5
    BIN_ID  =  6
    # number_of_best_fit = compute_number_of_fit_with_rotation( bin_size,item_size)
    M_lwh= number_of_best_fit[0]
    M_l= number_of_best_fit[1]
    M_w= number_of_best_fit[2]
    M_h= number_of_best_fit [3]
    best_rotate = number_of_best_fit[4]
    # print("number_of_best_fit",number_of_best_fit)
    OZ_rotation_lock= item_size[9]
    rotate_size = compute_rotate_item_size(item_size, best_rotate, OZ_rotation_lock)
    # print("rotate_size", rotate_size)
    # number_item  =  item_size[ITEM_NUM] 
    if  number_item> M_lwh:
        number_item = M_lwh
    # item_size[3] =  item_size[3]- number_item
    num_of_face= math.floor(number_item/(M_w*M_h)) 
    # số lát bánh mỳ
    for i in range(num_of_face):
        for j in range(M_w):
            a= rotate_size[ITEM_L_ID]
            b= rotate_size[ITEM_W_ID]
            c= rotate_size[ITEM_H_ID]*M_h

            d= M_h
            e= best_rotate #item_size[ITEM_ROTATE]

            f= bin_size[BIN_POS_L_ID]+ i*rotate_size[ITEM_L_ID]
            g= bin_size[BIN_POS_W_ID]+ j*rotate_size[ITEM_W_ID]
            h= bin_size[BIN_POS_H_ID]

            s= rotate_size[4]
            mylist.append( np.array([a, b, c, d, e, f, g, h, s], np.int_))

    left_over_of_face= number_item%(M_w*M_h)
    num_of_column = math.floor(left_over_of_face/M_h)
    for j in range(num_of_column):

        a= rotate_size[ITEM_L_ID]
        b= rotate_size[ITEM_W_ID]
        c= rotate_size[ITEM_H_ID] *M_h

        d= M_h
        e= best_rotate #item_size[ITEM_ROTATE]

        f= bin_size[BIN_POS_L_ID]+ num_of_face*rotate_size[ITEM_L_ID]
        # if f==    3790:
        #     print("fuck_________________________")
        #     print(bin_size[BIN_POS_L_ID], num_of_face, item_size[ITEM_L_ID])
        g= bin_size[BIN_POS_W_ID]+ j*rotate_size[ITEM_W_ID]
        h= bin_size[BIN_POS_H_ID]

        s= rotate_size[4]
        mylist.append( np.array([a, b, c, d, e, f, g, h, s], np.int_))

    left_over_of_column = number_item % M_h
    if left_over_of_column>0:
        a= rotate_size[ITEM_L_ID]
        b= rotate_size[ITEM_W_ID]
        c= rotate_size[ITEM_H_ID]*left_over_of_column

        d= left_over_of_column
        e= best_rotate #item_size[ITEM_ROTATE]

        f= bin_size[BIN_POS_L_ID]+ num_of_face*rotate_size[ITEM_L_ID]
        g= bin_size[BIN_POS_W_ID]+ num_of_column*rotate_size[ITEM_W_ID]
        h= bin_size[BIN_POS_H_ID]

        s= rotate_size[4]
        mylist.append( np.array([a, b, c, d, e, f, g, h, s], np.int_))

    return mylist


    


In [ ]:
# class Constant3D:
#     def __init__(self):
#         self.ROTATE_FIX =  np.array([  [0,1,2], [0,2,1], [1,0,2],[1,2,0], [2,1,0],[2,0,1]], np.int_)
#         self.BIN_L_ID  =  0
#         self.BIN_W_ID  =  1
#         self.BIN_H_ID  =  2
#         self.BIN_POS_L_ID  = 3
#         self.BIN_POS_W_ID  =  4
#         self.BIN_POS_H_ID  =  5
#         self.BIN_ID  =  6

#         self.ITEM_L_ID  =  0
#         self.ITEM_W_ID  =  1
#         self.ITEM_H_ID  =  2
#         self.ITEM_NUM  =  3
#         self.ITEM_ROTATE =  4
#         self.ITEM_POS_L_ID  = 5
#         self.ITEM_POS_W_ID  =  6
#         self.ITEM_POS_H_ID  = 7
#         self.ITEM_ID  =  8
#         self.DATA_SIZE = 5    
    
    
class Packing:
    def __init__(self, bin_list, item_list):
        self.bin_list = bin_list
        self.item_list = item_list
        self.original_item_list = np.copy(item_list)
        self.pre_packed_item = []
        self.original_bin_list = np.copy(bin_list)


    def pack( self, pre_packed_item, left_over_bin, can_not_pact_bin):
        bin_list = self.bin_list
        item_list = self.item_list
        min_item_dim = min([ min(item[0], item[1],item[2]) for item in item_list])
        lock_bin = []
        # original_cbm = check_cbm_bin_list(bin_list)
        move_bin_to_next_item  = []
        
        for i in range(len(item_list)):
            # print("________________")
            # print("pack item", i)
            item  =  item_list[i]
            # print('item to pack', item)
            # print(bin_list)
            # print("total_cbm", check_cbm_bin_list(bin_list) + check_cbm_item(pre_packed_item)+check_cbm_bin_list(lock_bin))

            count = 0
            if len(bin_list )==0:
                break
            b = len(bin_list )-1
            move_bin_to_next_item  = []
            while True:
                b = len(bin_list )-1

                if len(bin_list )==0:
                    # print("\t\t0, hết bin")
                    break       
                if b == len(bin_list) or b<0:
                    # print("\t\t1, hết bin")
                    break
                # print("\tlen(bin_list)", len(bin_list))
                temp_bin_list = []
                # print("\tpack item",i, " to bin ",b, "in",len(bin_list), " bins" )
                count= count+1
                # bin_list.sort(key= lambda x: -min( [x[0], x[1],x[2] ] ) )
                bin_list.sort(key= lambda x: -(x[0]**2-x[2]) )

                # err = original_cbm-check_cbm_bin_list(bin_list)-\
                # check_cbm_item(pre_packed_item)- check_cbm_bin_list(lock_bin) - check_cbm_bin_list(move_bin_to_next_item)
                # print( "\t",original_cbm/10**9, "space to be packed", check_cbm_bin_list(bin_list)/10**9, "packed cbm", check_cbm_item(pre_packed_item)/10**9, "unusable space",  check_cbm_bin_list(lock_bin)/10**9, check_cbm_bin_list(move_bin_to_next_item)/10**9 )
                # # if err!=0:
                # #     print("error", err)
                
                bin = bin_list[b]
                # print('\t\tbin to bin pack', bin)

    # lock_bin
                if min(bin[0],bin[1], bin[2] ) < min_item_dim:
                    lock_bin.append(bin)
                    bin_list.pop(b)
                    # print("\t\t2, không vừa, khóa bin, move to next bin")
                    # bin_list= bin_list+ temp_bin_list
                    continue
    # move_bin_to_next_item
                if item[3] == 0:
                    # print("\t\t3, hết item")
                    move_bin_to_next_item.append(bin)

                    bin_list.pop(b)
                    # bin_list= bin_list+ temp_bin_list
                    break

    # move_bin_to_next_item

                if min(bin[0], bin[1], bin[2]) < min( [item[0], item[1], item[2] ]) :
                    move_bin_to_next_item.append(bin)
                    # cai nay, can cho vao next round
                    # print("\t\t4, khong vừa minsize, move to next bin")
                    bin_list.pop(b)
                    # bin_list= bin_list+ temp_bin_list

                    continue


                number_of_best_fit = compute_number_of_fit_with_rotation( bin, item)
                number_of_fit  = number_of_best_fit[0]
    # move_bin_to_next_item
                if number_of_fit == 0:
                    move_bin_to_next_item.append(bin)
                    
                    bin_list.pop(b)
                    # print("\t\t5, khong vừa, move to next bin")
                    # bin_list= bin_list+ move_bin_to_next_item

                    continue
                else:
                    box  =  helper.decompose_bin( bin, item) 
                    for k in range(len(box)):
    # lock_bin
                        if min(box[k][0], box[k][1], box[k][2]) <1:
                            # lock_bin.append(  box[k])
                            continue
    # lock_bin
                        if min(box[k][0], box[k][1], box[k][2]) < min_item_dim :
                            lock_bin.append(  box[k])
                            continue

                        temp_bin_list.append(box[k])
                    move_item_to_prepack(bin, item, number_of_best_fit, pre_packed_item)
                    bin_list.pop(b)
                    # print("\t\t6 pack được tối đa", number_of_fit, "item, insert bin phụ", len(temp_bin_list))
                    # print(item)
                    bin_list= bin_list+ temp_bin_list

                    continue
                # print(len(bin_list), len(temp_bin_list))
            bin_list= bin_list+ move_bin_to_next_item
        self.lock_bin= lock_bin
        self.bin_list =  bin_list 
        self.pre_packed_item = pre_packed_item

        return     lock_bin, bin_list, pre_packed_item
    def decompress(self):
        pre_packed_item= self.pre_packed_item
        mylist = List()
        mylist.append(np.array([0,0,0,0,    0,0,0,0,0], np.int_))
        mylist.pop(0)
        for bunch in pre_packed_item:
            
            bin, item, num_item, number_of_best_fit = bunch
            mylist = insert___( bin, item, mylist, num_item, number_of_best_fit)
        
        mylist_final = [ [ item[0],item[1], item[2], item[5], item[6], item[7]] for item in mylist] 
        self.mylist =  mylist_final
        return mylist_final


    
    def draw_packed_bin(self, bin_size, item_list):
        def get_random_color():
            return "#%06x" % rd.randint(0, 0xFFFFFF)
        color_list  =  []
        for i in range(1000):
            color_list.append(get_random_color())


        data  =  []
        
        data.append(
            go.Mesh3d(
                x = [0, 0, int(bin_size[0]), int(bin_size[0]), 0, 0, int(bin_size[0]), int(bin_size[0])],
                y = [0, int(bin_size[1]), int(bin_size[1]), 0, 0, int(bin_size[1]), int(bin_size[1]), 0],
                z = [0, 0, 0, 0, int(bin_size[2]), int(bin_size[2]), int(bin_size[2]), int(bin_size[2])],
                # Intensity of each vertex, which will be interpolated and color-coded  
                # i, j and k give the vertices of triangles

                # i  =  [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                # j  =  [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                # k  =  [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                name = 'y',
                showscale = True,
                color = "#555",
                opacity = 0.1
            )
        )

        for index in range(len((item_list))):
            item =  item_list[index]
            ox_item  =  int(item[3])
            oy_item  =  int(item[4])
            oz_item  =  int(item[5])

            # true_size  =  item.get_dimension()
            
            extreme_x  =  int(item[0]) + ox_item
            extreme_y  =  int(item[1]) + oy_item
            extreme_z  =  int(item[2]) + oz_item
            # print(oz_item,extreme_x)

            data.append(
                go.Mesh3d(
                    x = [ox_item, ox_item, extreme_x, extreme_x, ox_item, ox_item, extreme_x, extreme_x],
                    y = [oy_item, extreme_y, extreme_y, oy_item, oy_item, extreme_y, extreme_y, oy_item],
                    z = [oz_item, oz_item, oz_item, oz_item, extreme_z, extreme_z, extreme_z, extreme_z],
                    # Intensity of each vertex, which will be interpolated and color-coded  
                    # i, j and k give the vertices of triangles
                    i  =  [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                    j  =  [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                    k  =  [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                    # name   =    " %s " %(item.name) ,
                    showscale  =  True,
                    # color  = 'red',
                    color  =  get_random_color(),
                    hovertext  =  "box "+str(index),
                    opacity = 1, flatshading  =  True
                )
            )
            data.append(go.Scatter3d(x = [0], y = [0], z = [0], mode  =  'markers'))
    
        fig  =  go.Figure(data)
        # fig   =   go.Figure(data = [go.Mesh3d(x = [0], y = [0], z = [0], color = 'lightpink', opacity = 0.50)])
        fig.update_layout(
            autosize = False,
            width = 1300,
            height  =  900,
            scene_aspectmode  =  'data', 
            scene_camera  =  dict( eye = dict(x = -2, y = -2, z = -1))
            )

        fig.show()

    def visualize_item(self):
        self.draw_packed_bin(self.original_bin_list[0], self.mylist)


    @staticmethod
    @njit (cache= True)
    def compute_number_of_fit(bin_size,item_size): #find number of fit of an item without rotate.
        M_l = math.floor(bin_size[0]/item_size[0])
        M_w = math.floor(bin_size[1]/item_size[1])
        M_h = math.floor(bin_size[2]/item_size[2])
        result  =  np.empty((4), np.int_)
        result[0] =  M_h*M_l*M_w
        result[1] =  M_l
        result[2] =  M_w
        result[3] =  M_h
        return result

    

In [ ]:
class Bin:
    def __init__(self, size):
        self.size  = size
 
    # def print(self):
        # print("size", self.size)
class Item:
    def __init__(self, size):
        self.size =  size
    # def print(self):
        # print("data", self.size)




# Calling 

In [ ]:

def create_data_model():
    bin_list_obj  =  []
    bin_list_obj.append(Bin(np.array([5890, 2330,2380, 0, 0, 0,   0, 0], np.int_) ))
    # bin_list_obj.append(Bin(np.array([1001, 1001,1001, 0, 0, 0,   0, 0], np.int_) ))
    bin_list  = [bin.size for bin in bin_list_obj]
    bin_size  =   bin_list[0]
    item_list_obj = []
    cbm_all_item  = 0
    bin_cbm =bin_size[0]*bin_size[1]*bin_size[2]

    num_item =0
    for i in range(10000):
        a,b,c,d = rd.randint(500, 50*i+ 500), rd.randint(500,50*i+500), rd.randint(50,500), 100
        # a,b,c,d = 300+3*i, 240-3*i, 315-3*i, 100+i
        num_item+=d
        cbm_all_item= cbm_all_item + a*b*c*d

        item_list_obj.append(Item(np.array([a,b,c,d   ,0, 0, 0, 0,     0, rd.randint(0,1)], np.int_)))
        
#         self.ITEM_L_ID  =  0
#         self.ITEM_W_ID  =  1
#         self.ITEM_H_ID  =  2
#         self.ITEM_NUM  =  3

#         self.ITEM_ROTATE =  4
#         self.ITEM_POS_L_ID  = 5
#         self.ITEM_POS_W_ID  =  6
#         self.ITEM_POS_H_ID  = 7


#         self.ITEM_ID  =  8
#         self.OZ_LOCK = 9    
    
        if bin_cbm<cbm_all_item:
            break
    item_list  =  [item.size for item in item_list_obj]
    item_list.sort(key= lambda x: -max([x[0], x[1],x[2]]))
    print(num_item)
    return bin_list, item_list
bin_list, item_list = create_data_model()

300


##testing

In [ ]:

# packing_problem.compute_rotate_item_size(packing_problem.item_list[0], 3)

In [ ]:
def packing(bin_list, item_list):
    bin_list_cbm= bin_list[0][0]*bin_list[0][1]*bin_list[0][2]/10**9
    original_item_cbm =sum([s[0]*s[1]*s[2]*s[3] for s in  item_list])/10**9
    packing_problem = Packing(bin_list, item_list)
    data={}
    data['original_bin'] = np.copy(bin_list[0])

    pre_packed_item=[]
    left_over_bin=[]
    can_not_pact_bin=[]

    temp_bin_list  =   []
    left_over_bin = []
    pre_packed_item = []
    left_over_item  = []

    result =packing_problem.pack(pre_packed_item, left_over_bin, can_not_pact_bin) #lock_bin, bin_list, pre_packed_item

    left_over_item_cbm =sum([s[0]*s[1]*s[2]*s[3] for s in  item_list]) /10**9
    data['left_over_item_cbm'] =left_over_item_cbm
    data['ratio'] =    (original_item_cbm- left_over_item_cbm)/ (bin_list_cbm)

    print("left_over_item_list", item_list)
    return packing_problem, data, result, left_over_item_cbm<1
    
time1=time.time()
packing_problem, data, result, fit_or_not = packing(*create_data_model())
time.time()-time1

500
left_over_item_list [array([683, 569, 328,   0,   0,   0,   0,   0,   0,   0]), array([512, 606,  58,   0,   0,   0,   0,   0,   0,   1]), array([595, 566, 133,   0,   0,   0,   0,   0,   0,   0]), array([515, 533, 430,  20,   0,   0,   0,   0,   0,   1]), array([500, 500, 319, 100,   0,   0,   0,   0,   0,   0])]


3.1683385372161865

In [ ]:
packing_problem.pre_packed_item

[[array([5890, 2330, 2380,    0,    0,    0,    0,    0]),
  array([683, 569, 328, 100,   0,   0,   0,   0,   0,   0]),
  100,
  array([224,   8,   4,   7,   0])],
 [array([ 683,  623, 2380, 2049, 1707,    0,    1,    0]),
  array([512, 606,  58,  41,   0,   0,   0,   0,   0,   1]),
  41,
  array([41,  1,  1, 41,  0])],
 [array([ 683,  569, 1724, 2049, 1138,  656,    2,    0]),
  array([512, 606,  58,  29,   0,   0,   0,   0,   0,   1]),
  29,
  array([29,  1,  1, 29,  1])],
 [array([ 683, 1138,   84, 2049,    0, 2296,    3,    0]),
  array([512, 606,  58,   2,   0,   0,   0,   0,   0,   1]),
  2,
  array([2, 1, 2, 1, 1])],
 [array([2049, 2276,   84,    0,    0, 2296,    5,    0]),
  array([512, 606,  58,  12,   0,   0,   0,   0,   0,   1]),
  12,
  array([12,  4,  3,  1,  0])],
 [array([3158, 2330, 2380, 2732,    0,    0,    0,    0]),
  array([512, 606,  58,  16,   0,   0,   0,   0,   0,   1]),
  16,
  array([820,   5,   4,  41,   1])],
 [array([ 171,  623, 2380, 2561, 1707,    0,   

In [ ]:
1/data['ratio']

1.1473290198391872

In [ ]:
rd.randint(0,1)

0

0.8899204267438228

In [ ]:
mylist =packing_problem.decompress()
packing_problem.draw_packed_bin(packing_problem.original_bin_list[0], packing_problem.mylist)


In [ ]:
mylist

[[683, 569, 2296, 0, 0, 0],
 [683, 569, 2296, 0, 569, 0],
 [683, 569, 2296, 0, 1138, 0],
 [683, 569, 2296, 0, 1707, 0],
 [683, 569, 2296, 683, 0, 0],
 [683, 569, 2296, 683, 569, 0],
 [683, 569, 2296, 683, 1138, 0],
 [683, 569, 2296, 683, 1707, 0],
 [683, 569, 2296, 1366, 0, 0],
 [683, 569, 2296, 1366, 569, 0],
 [683, 569, 2296, 1366, 1138, 0],
 [683, 569, 2296, 1366, 1707, 0],
 [683, 569, 2296, 2049, 0, 0],
 [683, 569, 2296, 2049, 569, 0],
 [683, 569, 656, 2049, 1138, 0],
 [512, 606, 2378, 2049, 1707, 0],
 [606, 512, 1682, 2049, 1138, 656],
 [606, 512, 58, 2049, 0, 2296],
 [606, 512, 58, 2049, 512, 2296],
 [512, 606, 58, 0, 0, 2296],
 [512, 606, 58, 0, 606, 2296],
 [512, 606, 58, 0, 1212, 2296],
 [512, 606, 58, 512, 0, 2296],
 [512, 606, 58, 512, 606, 2296],
 [512, 606, 58, 512, 1212, 2296],
 [512, 606, 58, 1024, 0, 2296],
 [512, 606, 58, 1024, 606, 2296],
 [512, 606, 58, 1024, 1212, 2296],
 [512, 606, 58, 1536, 0, 2296],
 [512, 606, 58, 1536, 606, 2296],
 [512, 606, 58, 1536, 1212, 22